In [6]:
import pandas as pd
import pulp

# Chargement des données (stock et capacités des noeuds et arêtes)
nodes_data = {
    'node': ['A', 'B', 'C'],
    'stock': [80, 150, 200],
    'capacity': [100, 200, 250]
}

edges_data = {
    'start': ['A', 'A', 'B'],
    'end': ['B', 'C', 'C'],
    'capacity': [50, 60, 70]  # capacité maximale de chaque arête
}

nodes_df = pd.DataFrame(nodes_data)
edges_df = pd.DataFrame(edges_data)

# Création du modèle d'optimisation
model = pulp.LpProblem("Optimisation_des_envois", pulp.LpMinimize)

# Création des variables de décision pour chaque paire de noeuds (start, end) : le nombre de palettes à envoyer
variables = {}
for index, edge in edges_df.iterrows():
    i, j = edge['start'], edge['end']
    variables[(i, j)] = pulp.LpVariable(f"x_{i}_{j}", lowBound=0, cat='Continuous')

# Variables pour les écarts absolus (pour l'objectif)
deviation_stock = {}
deviation_movement = {}

# Contraintes de stock pour chaque noeud
for index, node in nodes_df.iterrows():
    i = node['node']
    stock_initial = node['stock']
    capacity = node['capacity']
    
    # Calcul du stock après mouvements (sorties - entrées)
    outflow = pulp.lpSum(variables[(i, j)] for (i_, j) in variables if i_ == i)
    inflow = pulp.lpSum(variables[(j, i)] for (j, i_) in variables if i_ == i)
    final_stock = stock_initial - outflow + inflow
    
    # Contraintes : le stock final doit être entre 70% et 100% de la capacité du noeud
    model += (final_stock >= 0.7 * capacity, f"Stock_min_{i}")
    model += (final_stock <= capacity, f"Stock_max_{i}")
    
    # Variable auxiliaire pour l'écart absolu par rapport à 85% de la capacité
    deviation_stock[i] = pulp.LpVariable(f"deviation_stock_{i}", lowBound=0, cat='Continuous')
    
    # Contrainte d'écart absolu pour atteindre 85% de la capacité
    model += (final_stock - 0.85 * capacity <= deviation_stock[i], f"Stock_dev_sup_{i}")
    model += (0.85 * capacity - final_stock <= deviation_stock[i], f"Stock_dev_inf_{i}")

# Contraintes de mouvement pour chaque arête
for index, edge in edges_df.iterrows():
    i, j = edge['start'], edge['end']
    capacity = edge['capacity']
    
    # Mouvement total sur l'arête (i, j) est simplement le nombre de palettes envoyées sur cette arête
    total_movement = variables[(i, j)]
    
    # Contraintes : le mouvement total doit être entre 70% et 100% de la capacité de l'arête
    model += (total_movement >= 0.7 * capacity, f"Movement_min_{i}_{j}")
    model += (total_movement <= capacity, f"Movement_max_{i}_{j}")
    
    # Variable auxiliaire pour l'écart absolu par rapport à 85% de la capacité de l'arête
    deviation_movement[(i, j)] = pulp.LpVariable(f"deviation_movement_{i}_{j}", lowBound=0, cat='Continuous')
    
    # Contrainte d'écart absolu pour atteindre 85% de la capacité de l'arête
    model += (total_movement - 0.85 * capacity <= deviation_movement[(i, j)], f"Movement_dev_sup_{i}_{j}")
    model += (0.85 * capacity - total_movement <= deviation_movement[(i, j)], f"Movement_dev_inf_{i}_{j}")

# Minimisation de la somme des écarts par rapport aux cibles de 85%
model += pulp.lpSum(deviation_stock[i] for i in nodes_df['node']) + \
         pulp.lpSum(deviation_movement[(i, j)] for (i, j) in variables)

# Résolution du modèle
model.solve()

# Extraction des résultats optimaux
if pulp.LpStatus[model.status] == 'Optimal':
    print("Solution optimale trouvée :")
    results = []
    for (i, j), var in variables.items():
        print(f"Envoyer {var.value()} palettes de {i} vers {j}")
        results.append({'Départ': i, 'Arrivée': j, 'Nombre de palettes': var.value()})
        
    # Exporter les résultats en CSV
    results_df = pd.DataFrame(results)
    results_df.to_csv('envois_optimaux.csv', index=False)
    print("Fichier CSV des envois optimaux généré.")
else:
    print("Aucune solution optimale n'a été trouvée.")



Aucune solution optimale n'a été trouvée.
